In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from model import *
from data import prepare_data_vocab, live_feat
from torch.nn.utils.rnn import unpack_sequence
from torch_geometric.data import Data


import torch.optim as optim
from torch_geometric.data import Data

ImportError: /home/reza/.local/lib/python3.10/site-packages/torch/lib/../../nvidia/cusparse/lib/libcusparse.so.12: undefined symbol: __nvJitLinkAddData_12_1, version libnvJitLink.so.12

In [17]:
word_to_idx, idx_to_word, data, data_emb = prepare_data_vocab("data", func=live_feat, function_num=0)

In [14]:
cnt = 0
for i in idx_to_word.keys():
    print(idx_to_word[i], end='\t|')
    cnt+= 1

print("\n----------\n",cnt)

(INFUNCTION_VAR * INFUNCTION_VAR)TMP_FUNCTION_NAME	|(INFUNCTION_VAR *)TMP_VAR	|*	|+	|-	|-TMP_NUMBER	|-TMP_VAR	|/	|;	|INFUNCTION_VAR	|INFUNCTION_VAR.INFUNCTION_VAR	|TMP_FUNCTION_ARG_1	|TMP_FUNCTION_ARG_10	|TMP_FUNCTION_ARG_11	|TMP_FUNCTION_ARG_2	|TMP_FUNCTION_ARG_3	|TMP_FUNCTION_ARG_4	|TMP_FUNCTION_ARG_5	|TMP_FUNCTION_ARG_6	|TMP_FUNCTION_ARG_7	|TMP_FUNCTION_ARG_8	|TMP_FUNCTION_ARG_9	|TMP_FUNCTION_NAME	|TMP_NUMBER	|TMP_POINTER_MEMBER	|TMP_VAR	|TMP_VAR(INFUNCTION_VAR)	|TMP_VAR(TMP_FUNCTION_NAME)	|assignment	|char	|double	|float	|function_call	|gsl_error	|int	|load	|math_op	|phi	|size_t	|store	|
----------
 40


In [91]:
#define InputEncoder (RNN) and GNN
in_dim = 8
hidden_dim = 64
num_layers = 7
bidirectional = True
out_dim    = 128

int_dir = 2 if bidirectional else 1

bb_enc = InputEncoder(word_to_idx, 
                      in_dim=in_dim, 
                      hidden_dim=hidden_dim, 
                      num_layers=num_layers, 
                      bidirectional=bidirectional)
 
gnn = GNNModel(in_dim=int_dir*hidden_dim,
               out_dim=out_dim,
               layer_dims=[16, 32])

In [4]:
in_dim = 8
hidden_dim = 16
num_layers = 7
bidirectional = True
out_dim    = 32

int_dir = 2 if bidirectional else 1
integrated = IntegratedModel(word_to_idx, 
                      in_dim_lstm=in_dim, 
                      hidden_dim=hidden_dim, 
                      num_layers=num_layers, 
                      bidirectional=bidirectional,
                        out_dim=out_dim,
                        layer_dims=[16, 32])

In [6]:
assert len(data)==len(data_emb)
for i in range(len(data)):
    assert len(data[i][2])==len(data_emb[i][1])

#define InputEncoder (RNN) and GNN
in_dim = 8
hidden_dim = 16
num_layers = 7
bidirectional = True
out_dim    = 16

int_dir = 2 if bidirectional else 1

bb_enc = InputEncoder(word_to_idx, 
                      in_dim=in_dim, 
                      hidden_dim=hidden_dim, 
                      num_layers=num_layers, 
                      bidirectional=bidirectional)
 
gnn = GNNModel(in_dim=int_dir*hidden_dim,
               out_dim=out_dim,
               layer_dims=[16, 32])

In [38]:
initial_node_embeddings = []
new_data_emb = []
for graph in data_emb:
    edge_index, node_embs = graph
    #if torch.tensor(edge_index).shape[0]==0:
    if len(edge_index) != 0:
        edge_index = torch.tensor(edge_index).T
    else:
        edge_index = torch.tensor([[0],[0]])
    
    out, h, c = bb_enc(node_embs)
    node_feats = torch.vstack([k[-1] for k in unpack_sequence(out)])

    assert node_feats.shape == (len(node_embs), int_dir*hidden_dim)

            #Step 2: gnn forward pass
    graph = Data(edge_index=edge_index,
                        x=node_feats)
    initial_node_embeddings.append(torch.rand((graph.num_nodes, out_dim), requires_grad=True))
    new_data_emb.append([edge_index, node_embs, initial_node_embeddings])

In [28]:
edge_index, node_embs = data_emb[0]
#if torch.tensor(edge_index).shape[0]==0:

if len(edge_index) != 0:
    edge_index = torch.tensor(edge_index).T
else:
    edge_index = torch.tensor([[0],[0]])
            
        
out, h, c = bb_enc(node_embs)

In [102]:
hidden_dim = 13
ae = AE_rnn(word_to_idx, 7, hidden_dim, 3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ae.parameters(), lr=0.01)
epoch_num = 1000
for epoch in range(epoch_num):
    total_loss = 0
    total_items = 0
    j = -1
    for graph in data_emb:
        j += 1
        edge_index, node_embs = graph
        for bb in node_embs:

            out, embs = ae(bb)
            #print(a.shape)
            #print(out)
            #total_loss += unsupervised_loss.item()
            #loss = criterion(out, embs)
            loss = criterion(out, bb)
            #loss = criterion(output, initial_node_embeddings[j])
            # Backward pass and optimization
            total_loss  += loss.item()
            total_items += bb.shape[0]

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    # Print or log the average loss for monitoring
    if epoch%10==0 or epoch==epoch_num-1:
        avg_loss = total_loss / len(data_emb)
        print(f'For {j+1} functions: Epoch {epoch+1}/{epoch_num}, Average Loss: {avg_loss:.5f}', end='\r')
    if epoch%100==0 or epoch==epoch_num-1:
        avg_loss = total_loss / len(data_emb)
        print(f'For {j+1} functions: Epoch {epoch+1}/{epoch_num}, Average Loss: {avg_loss:.5f}', end='\n')

For 5 functions: Epoch 1/1000, Average Loss: 30.03522


KeyboardInterrupt: 

In [129]:
torch.cuda.is_available()

True

In [131]:
hidden_dim = 13
ae = AE_gnnrnn(word_to_idx, 7, hidden_dim, 3)

device = "cuda" 
ae_gpu  = ae.to(device)

In [133]:
data_emb_gpu = torch.stack(data_emb).to(device)

TypeError: expected Tensor as element 0 in argument 0, but got tuple

In [26]:
hidden_dim = 13
ae = AE_gnnrnn(word_to_idx, 7, hidden_dim, 3)

#device = "cuda" 
#ae = ae.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ae.parameters(), lr=0.01)
epoch_num = 1000
for epoch in range(epoch_num):
    total_loss = 0
    total_items = 0
    j = -1
    
    for graph in data_emb:
        j += 1
        edge_index, node_embs = graph
        #edge_index = edge_index.cuda()
        #node_embs_gpu = [emb.cuda() for emb in node_embs]
        #outs = ae(node_embs_gpu, edge_index)
        outs = ae(node_embs, edge_index)
        i = -1
        for bb in node_embs:
            i += 1
            
            loss = criterion(outs[i], bb)
            #print(outs[i].shape)
            #print(bb.shape)
            #loss = criterion(output, initial_node_embeddings[j])

            # Backward pass and optimization
            total_loss  += loss.item()
            total_items += bb.shape[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Print or log the average loss for monitoring
    if epoch%10==0 or epoch==epoch_num-1:
        avg_loss = total_loss / len(data_emb)
        print(f'For {j+1} functions: Epoch {epoch+1}/{epoch_num}, Average Loss: {avg_loss:.5f}', end='\r')
    if epoch%100==0 or epoch==epoch_num-1:
        avg_loss = total_loss / len(data_emb)
        print(f'For {j+1} functions: Epoch {epoch+1}/{epoch_num}, Average Loss: {avg_loss:.5f}', end='\n')

For 578 functions: Epoch 1/1000, Average Loss: 16.72916


KeyboardInterrupt: 

In [28]:
hidden_dim = 13
ae = AE_gnnrnn_gpu(word_to_idx, 7, hidden_dim, 3)

#device = "cuda" 
#ae.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ae.parameters(), lr=0.01)
epoch_num = 1000
for epoch in range(epoch_num):
    total_loss = 0
    total_items = 0
    j = -1
    
    for graph in data_emb:
        j += 1
        edge_index, node_embs = graph
        outs = ae(node_embs, edge_index)
        i = -1
        for bb in node_embs:
            i += 1
            
            loss = criterion(outs[i], bb)
            #print(outs[i].shape)
            #print(bb.shape)
            #loss = criterion(output, initial_node_embeddings[j])

            # Backward pass and optimization
            total_loss  += loss.item()
            total_items += bb.shape[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Print or log the average loss for monitoring
    if epoch%10==0 or epoch==epoch_num-1:
        avg_loss = total_loss / len(data_emb)
        print(f'For {j+1} functions: Epoch {epoch+1}/{epoch_num}, Average Loss: {avg_loss:.5f}', end='\r')
    if epoch%100==0 or epoch==epoch_num-1:
        avg_loss = total_loss / len(data_emb)
        print(f'For {j+1} functions: Epoch {epoch+1}/{epoch_num}, Average Loss: {avg_loss:.5f}', end='\n')

AttributeError: 'Tensor' object has no attribute 'batch_sizes'

In [96]:
random_target = torch.rand((len(data_emb), 1, out_dim))

In [97]:
random_target.shape

torch.Size([578, 1, 32])

In [16]:
data_emb[0][1]

[tensor([ 8, 28, 25, 23,  8,  8,  8,  8,  8, 32,  9, 11, 14, 15,  8,  8, 36, 25,
         25,  4, 23,  8,  8, 36, 25, 25,  7, 23,  8,  8,  8,  8,  8]),
 tensor([ 8, 36, 25, 25,  3, 23,  8,  8]),
 tensor([ 8, 37,  8,  8, 36, 25, 25,  3, 23,  8,  8, 36, 25, 25,  7, 23,  8,  8,
          8,  8,  8]),
 tensor([ 8, 36, 25, 25,  3, 23,  8,  8]),
 tensor([ 8, 37,  8,  8, 36, 25, 25,  4, 23,  8,  8, 36, 25, 25,  7, 23,  8,  8,
          8,  8,  8]),
 tensor([ 8, 36, 25, 25,  3, 23,  8,  8]),
 tensor([ 8, 37,  8, 32,  9, 11, 14, 15,  8,  8, 36, 25, 25,  2, 23,  8,  8,  8,
         36, 25, 25,  2, 23,  8,  8,  8, 36, 25, 25,  2, 23,  8,  8,  8, 36, 25,
         25,  4, 23,  8,  8,  8,  8,  8,  8]),
 tensor([ 8, 36, 25, 25,  3, 23,  8,  8]),
 tensor([ 8, 37,  8,  8, 36, 25, 25,  4, 23,  8,  8, 36, 25, 25,  7, 23,  8,  8,
          8,  8,  8]),
 tensor([ 8, 36, 25, 25,  3, 23,  8,  8]),
 tensor([ 8, 37,  8,  8, 36, 25, 25,  4, 23,  8,  8, 36, 25, 25,  7, 23,  8,  8,
          8,  8,  8]),
 tensor(

In [8]:
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import DataLoader as GraphDataLoader
from torch.nn.utils.rnn import pad_sequence

class GraphDataset(Dataset):
    def __init__(self, graphs):
        self.graphs = graphs
        
    def __len__(self):
        return len(self.graphs)
    
    def __getitem__(self, idx):
        return self.graphs[idx]

def collate_fn(batch):
    # Extract node features and edge indices from the batch
    
    node_features = [graph[1] for graph in batch]
    edge_indices = [graph[0] for graph in batch]
    
    # Pad sequences to make them the same size within a batch
    node_features_padded = pad_sequence(node_features, batch_first=True, padding_value=0)
    edge_indices_padded = pad_sequence(edge_indices, batch_first=True, padding_value=-1)
    
    return node_features_padded, edge_indices_padded

In [41]:
from torch_geometric.data import DataLoader as GraphDataLoader
batch_size = 64
shuffle = True
num_workers = 4
train_loader = GraphDataLoader(data_emb, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
for batch in train_loader:
        node_features_batch, edge_indices_batch = batch

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/reza/.local/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/reza/.local/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
  File "/home/reza/.local/lib/python3.10/site-packages/torch_geometric/loader/dataloader.py", line 35, in __call__
    return [self(s) for s in zip(*batch)]
  File "/home/reza/.local/lib/python3.10/site-packages/torch_geometric/loader/dataloader.py", line 35, in <listcomp>
    return [self(s) for s in zip(*batch)]
  File "/home/reza/.local/lib/python3.10/site-packages/torch_geometric/loader/dataloader.py", line 23, in __call__
    return default_collate(batch)
  File "/home/reza/.local/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 265, in default_collate
    return col

In [44]:
batch_size = 64
shuffle = True
num_workers = 4
dataset    = GraphDataset(data_emb)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_fn)
for batch in dataloader:
        node_features_batch, edge_indices_batch = batch
        print(node_features_batch)

NameError: name 'GraphDataset' is not defined

In [5]:
in_dim = 8
hidden_dim = 16
num_layers = 1
bidirectional = True
out_dim    = 4

int_dir = 2 if bidirectional else 1
out_dim    = int(in_dim/int_dir)

model = AutoEncoder(word_to_idx, 
                    in_dim=in_dim, 
                    hidden_dim=hidden_dim, 
                    num_layers=num_layers, 
                    output_size=out_dim,
                    bidirectional=bidirectional)

optimizer = torch.optim.Adam(model.parameters())
torch.autograd.set_detect_anomaly(True)
criterion = nn.MSELoss()
num_epochs = 10
for epoch in range(num_epochs):
    for graph in data_emb:
        edge_index, node_embs = graph
        output, emb = model(node_embs)
        
        for i in range(len(emb)):
            for j in range(len(emb[i])):
                print(output[i][j].T[0])
                print(emb[i][j])

                optimizer.zero_grad()
                loss = criterion(output[i][j].T[0], emb[i][j])  # Compare with original sequence
                loss.backward()
                optimizer.step()

tensor([ 0.0322,  0.0419,  0.1197,  0.1828,  0.0559,  0.0648, -0.0141,  0.0522],
       grad_fn=<SelectBackward0>)
tensor([-0.3529,  0.2771,  0.2931,  1.6526, -1.7671, -0.0654, -1.2059, -1.6384],
       grad_fn=<SelectBackward0>)
tensor([ 0.0322,  0.0419,  0.1197,  0.1828,  0.0559,  0.0648, -0.0141,  0.0522],
       grad_fn=<SelectBackward0>)
tensor([-0.3529,  0.2771,  0.2931,  1.6526, -1.7671, -0.0654, -1.2059, -1.6384],
       grad_fn=<SelectBackward0>)


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [16, 8]] is at version 2; expected version 1 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!

In [16]:
https://github.com/TreeinRandomForest/gnnexamples.git

Min: -0.2046039701, Mean: -0.0094097275, Max: 0.1965838373 |#| For 5 functions: Epoch 1/100, Average Loss: 0.0005420736


KeyboardInterrupt: 

In [ ]:
gnn_output

tensor([[-0.0126,  0.0034,  0.1862,  0.0372,  0.0126,  0.1426, -0.0502,  0.0251,
          0.0074, -0.0424,  0.0644, -0.0057,  0.1713, -0.2475, -0.2109,  0.2017,
          0.0335,  0.0390, -0.0477, -0.0328,  0.0037,  0.0888,  0.0216, -0.0074,
          0.0678,  0.0013,  0.0570, -0.0472, -0.0675,  0.0315,  0.0022, -0.0574],
        [-0.0119,  0.0015,  0.1829,  0.0377,  0.0145,  0.1457, -0.0493,  0.0237,
          0.0046, -0.0451,  0.0696, -0.0094,  0.1666, -0.2515, -0.2126,  0.2000,
          0.0315,  0.0395, -0.0445, -0.0329,  0.0015,  0.0881,  0.0178, -0.0092,
          0.0670,  0.0021,  0.0529, -0.0489, -0.0652,  0.0246,  0.0071, -0.0584]],
       grad_fn=<AddmmBackward0>)

In [24]:
node_feats

tensor([[-7.3550e-03, -2.7184e-03,  1.9277e-01,  3.8857e-02, -2.6345e-03,
          1.4869e-01, -5.2959e-02,  2.5981e-02,  5.1645e-03, -3.0783e-02,
          6.8217e-02, -1.3734e-02,  1.8695e-01, -2.6015e-01, -2.3863e-01,
          2.4312e-01,  3.2007e-02,  4.0228e-02, -4.7154e-02, -3.0721e-02,
          2.4937e-03,  8.9804e-02,  1.9781e-02, -6.0409e-03,  6.5731e-02,
          4.7992e-03,  5.6716e-02, -4.7487e-02, -6.8936e-02,  3.0280e-02,
          4.6410e-03, -5.6473e-02],
        [-1.0848e-02,  8.0277e-03,  1.6795e-01,  3.6411e-02,  3.1916e-02,
          1.3602e-01, -5.0221e-02,  2.0493e-02,  5.5699e-03, -5.6578e-02,
          6.6495e-02, -2.0384e-03,  1.5436e-01, -2.4501e-01, -1.9379e-01,
          1.5640e-01,  3.2156e-02,  4.0920e-02, -4.6510e-02, -2.9881e-02,
         -1.8220e-04,  8.8291e-02,  1.9627e-02, -9.0167e-03,  6.8013e-02,
         -6.0287e-04,  5.4320e-02, -4.9530e-02, -6.3678e-02,  2.4718e-02,
          3.4426e-03, -6.2146e-02]])

In [128]:
min = 10000
max = 0
for graph in data_emb:
    edge_index, node_embs = graph
    if len(edge_index) != 0:
        if len(node_embs)>max:
            max = len(node_embs)
        if len(node_embs)<min:
            min = len(node_embs)
print("Max: ", max)
print("Min: ", min)

Max:  128
Min:  2


In [21]:
criterion = nn.MSELoss()
optimizer = optim.Adam(gnn.parameters(), lr=0.01)

epoch_num = 300
# Training loop

random_target = torch.rand((len(data_emb), 1, out_dim))
stop = 0
for epoch in range(epoch_num):
    total_loss = 0
    total_items = 0
    j = -1
    for graph in data_emb:
        j += 1
        edge_index, node_embs = graph
        #if torch.tensor(edge_index).shape[0]==0:
        if len(edge_index) != 0:
            edge_index = torch.tensor(edge_index).T
        else:
            edge_index = torch.tensor([[0],[0]])
            
        
        out, h, c = bb_enc(node_embs)
        node_feats = torch.vstack([k[-1] for k in unpack_sequence(out)])

        assert node_feats.shape == (len(node_embs), int_dir*hidden_dim)

                #Step 2: gnn forward pass
        graph = Data(edge_index=edge_index,
                            x=node_feats)
        
        output = gnn(graph)
        
        # Unsupervised loss (dummy loss for illustration)
        loss = criterion(output, random_target[j])
        #loss = criterion(output, initial_node_embeddings[j])

        # Backward pass and optimization
        total_loss  += loss.item()
        total_items += graph.x.shape[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch%10==0 or epoch==epoch_num-1:
            avg_loss = total_loss / len(data_emb)
            print(f'For {j+1} functions: Epoch {epoch+1}/{epoch_num}, Average Loss: {avg_loss:.5f}', end='\r')

        #total_loss += unsupervised_loss.item()

    # Print or log the average loss for monitoring
    if epoch%10==0 or epoch==epoch_num-1:
        print()


NameError: name 'gnn' is not defined

In [115]:
node_feats

tensor([[-1.6942e-06,  3.6233e-06, -7.3788e-04, -4.4494e-05,  3.6724e-08,
         -7.6004e-01, -2.7473e-03,  5.2556e-04,  3.6020e-06, -4.5144e-05,
         -9.2002e-07,  6.8339e-02, -2.3965e-04,  1.6933e-03, -1.9910e-04,
         -2.3565e-04,  4.8033e-02,  9.4108e-02, -7.9327e-02,  1.0815e-01,
          1.6961e-02, -1.9245e-02, -3.6383e-03,  1.5153e-01, -4.3944e-02,
         -2.2803e-02, -7.2129e-02,  7.6061e-02, -3.1655e-02, -1.4341e-01,
          9.2256e-02, -1.8506e-01],
        [-1.6942e-06,  3.6233e-06, -7.3788e-04, -4.4494e-05,  3.6724e-08,
         -7.6004e-01, -2.7473e-03,  5.2556e-04,  3.6020e-06, -4.5144e-05,
         -9.2002e-07,  6.8339e-02, -2.3965e-04,  1.6933e-03, -1.9910e-04,
         -2.3565e-04,  4.8033e-02,  9.4108e-02, -7.9327e-02,  1.0815e-01,
          1.6961e-02, -1.9245e-02, -3.6383e-03,  1.5153e-01, -4.3944e-02,
         -2.2803e-02, -7.2129e-02,  7.6061e-02, -3.1655e-02, -1.4341e-01,
          9.2256e-02, -1.8506e-01],
        [-1.6942e-06,  3.6233e-06, -7.37

In [18]:
for name, param in integrated.named_parameters():
    if param.grad is not None:
        print(f"{name} grad norm: {param.grad.norm()}")
    else:
        print(f"{name} grad is None")

lstm_encoder.emb.weight grad is None
lstm_encoder.lstm.weight_ih_l0 grad is None
lstm_encoder.lstm.weight_hh_l0 grad is None
lstm_encoder.lstm.bias_ih_l0 grad is None
lstm_encoder.lstm.bias_hh_l0 grad is None
lstm_encoder.lstm.weight_ih_l0_reverse grad is None
lstm_encoder.lstm.weight_hh_l0_reverse grad is None
lstm_encoder.lstm.bias_ih_l0_reverse grad is None
lstm_encoder.lstm.bias_hh_l0_reverse grad is None
lstm_encoder.lstm.weight_ih_l1 grad is None
lstm_encoder.lstm.weight_hh_l1 grad is None
lstm_encoder.lstm.bias_ih_l1 grad is None
lstm_encoder.lstm.bias_hh_l1 grad is None
lstm_encoder.lstm.weight_ih_l1_reverse grad is None
lstm_encoder.lstm.weight_hh_l1_reverse grad is None
lstm_encoder.lstm.bias_ih_l1_reverse grad is None
lstm_encoder.lstm.bias_hh_l1_reverse grad is None
lstm_encoder.lstm.weight_ih_l2 grad is None
lstm_encoder.lstm.weight_hh_l2 grad is None
lstm_encoder.lstm.bias_ih_l2 grad is None
lstm_encoder.lstm.bias_hh_l2 grad is None
lstm_encoder.lstm.weight_ih_l2_reverse 

In [100]:
for name, param in bb_enc.named_parameters():
    if param.grad is not None:
        print(f"{name} grad norm: {param.grad.norm()}")
    else:
        print(f"{name} grad is None")

emb.weight grad norm: 4.2484352889005095e-05
lstm.weight_ih_l0 grad norm: 0.0003254949115216732
lstm.weight_hh_l0 grad norm: 7.605506834806874e-05
lstm.bias_ih_l0 grad norm: 0.0003012344823218882
lstm.bias_hh_l0 grad norm: 0.0003012344823218882
lstm.weight_ih_l0_reverse grad norm: 0.00032709658262319863
lstm.weight_hh_l0_reverse grad norm: 7.603189442306757e-05
lstm.bias_ih_l0_reverse grad norm: 0.000313809810904786
lstm.bias_hh_l0_reverse grad norm: 0.000313809810904786
lstm.weight_ih_l1 grad norm: 0.000558680621907115
lstm.weight_hh_l1 grad norm: 0.00022624272969551384
lstm.bias_ih_l1 grad norm: 0.0010487716645002365
lstm.bias_hh_l1 grad norm: 0.0010487715480849147
lstm.weight_ih_l1_reverse grad norm: 0.0004790965758729726
lstm.weight_hh_l1_reverse grad norm: 0.00016279509873129427
lstm.bias_ih_l1_reverse grad norm: 0.0009165560477413237
lstm.bias_hh_l1_reverse grad norm: 0.0009165561059489846
lstm.weight_ih_l2 grad norm: 0.0014243311015889049
lstm.weight_hh_l2 grad norm: 0.000649143

In [51]:
for name, param in integrated.named_parameters():
    if not param.requires_grad:
        print("not", param)
    

AttributeError: 'generator' object has no attribute 'grad'

In [49]:
import re

def extract_a_op_b(input_string):
    

    # Find and replace expressions within parentheses with a placeholder
    placeholder = '__PLACEHOLDER__'
    flag_par = 0
    while '(' in input_string and ')' in input_string:
        flag_par = 1
        # Find the innermost expression within parentheses
        inner_expr = re.search(r'\([^)]*\)', input_string).group(0)
        
        # Replace the inner expression with a placeholder
        input_string = input_string.replace(inner_expr, placeholder)

    # Define the original pattern
    original_pattern = r"([-+]?\b\d+(\.\d*)?(e[+-]?\d+)?\b|\b\w+\b|\[\s*\d+(\s*,\s*\d+)*\s*\])\s*([+\-*/])\s*([-+]?\b\d+(\.\d*)?(e[+-]?\d+)?\b|\b\w+\b|\[\s*\d+(\s*,\s*\d+)*\s*\])"

    # Find the first match in the modified input string
    match = re.search(original_pattern, input_string)

    if match:
        # Extract a, op, b from the match
        a = match.group(1)
        op = match.group(5)
        b = match.group(6)

        # Replace the placeholder back with the original expression
        if flag_par:
            a = a.replace(placeholder, inner_expr)
            b = b.replace(placeholder, inner_expr)

        return a, op, b
    else:
        return None

# Example usage
input_string = "_1 * 5.0e-1"
result = extract_a_op_b(input_string)

if result:
    a, op, b = result
    print("a:", a)
    print("op:", op)
    print("b:", b)
else:
    print("No match found.")


a: _1
op: *
b: 5.0e-1


In [35]:
import numpy as np
vocab = np.array(['a', 'b', 'c', 'a'])
vocab = np.unique(vocab)
word_to_idx = dict(zip(vocab, np.arange(len(vocab))))
idx_to_word = dict(zip(np.arange(len(vocab)), vocab))

In [36]:
word_to_idx

{'a': 0, 'b': 1, 'c': 2}